# Imports iniciais

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['figure.figsize']  = (16, 10)
plt.rcParams['axes.labelsize']  = 20
plt.rcParams['axes.titlesize']  = 20
plt.rcParams['legend.fontsize'] = 20
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['lines.linewidth'] = 4

In [ ]:
leagueoflegends = pd.read_csv('/content/drive/MyDrive/LolDatabase/LeagueofLegends.csv',header=0)

# Limpando dados

Nessa sessão são criadas colunas para as ligas de cada time e selecionadas as colunas que realmente serão usadas nas análises



In [ ]:
# vamos criar um dataframe com todos as tags de times e as ligas de cada um

leagueoflegendsL = leagueoflegends[leagueoflegends['League'] != 'WC']
leagueoflegendsL = leagueoflegendsL[leagueoflegendsL['League'] != 'MSI']
leagueoflegendsL = leagueoflegendsL[leagueoflegendsL['League'] != 'RR']
leagueoflegendsL = leagueoflegendsL[leagueoflegendsL['League'] != 'IEM']

leagueoflegendsL = leagueoflegendsL['League'].unique()
leagueoflegendsL # array com todas as ligas

timesLiga = pd.DataFrame() # dataframe que associa os times às suas ligas

times_to_liga = {}

for i in (leagueoflegendsL):
  timesR = leagueoflegends[leagueoflegends['League'] == i]['redTeamTag'].unique()
  timesB = leagueoflegends[leagueoflegends['League'] == i]['blueTeamTag'].unique()
  timesR = pd.Series(timesR)
  timesB = pd.Series(timesB)
  times = pd.concat([timesB,timesR])
  times = times.unique()
  for j in (times):
    times_to_liga[j] = i
    timesLiga = timesLiga.append(pd.Series([j,i]), ignore_index=True)
timesLiga = timesLiga.rename(columns={0:'time', 1:'liga'})

timesLiga = timesLiga.dropna()
timesLiga = timesLiga.drop_duplicates(subset=['time'])

timesLiga.loc[timesLiga['time'] == 'ROX', 'liga'] = 'LCK'
timesLiga.loc[timesLiga['time'] == 'FB', 'liga'] = 'TCL'

times_to_liga['ROX'] = 'LCK'
times_to_liga['FB'] = 'TCL'
times_to_liga['SSW'] = 'LCK'
times_to_liga['SHR'] = 'LPL'
times_to_liga['EDG'] = 'LPL'
times_to_liga['OMG'] = 'LPL'
times_to_liga['ALL'] = 'EULCS'
times_to_liga['LMQ'] = 'LPL'
times_to_liga['NWS'] = 'LCK'
times_to_liga['IG'] = 'LPL'
times_to_liga['BKT'] = 'LMS'
times_to_liga['LGD'] = 'LPL'
times_to_liga['RNG'] = 'LPL'
times_to_liga['GAM'] = 'LMS'
times_to_liga['WE'] = 'LPL'

In [ ]:
leagueoflegends['blueTeamLeague'] = leagueoflegends['League']
leagueoflegends['redTeamLeague'] = leagueoflegends['League']
leagueoflegends.head(n=1)

,League,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff,goldblue,bKills,bTowers,bInhibs,bDragons,bBarons,bHeralds,goldred,rKills,rTowers,rInhibs,rDragons,rBarons,rHeralds,blueTop,blueTopChamp,goldblueTop,blueJungle,blueJungleChamp,goldblueJungle,blueMiddle,blueMiddleChamp,goldblueMiddle,blueADC,blueADCChamp,goldblueADC,blueSupport,blueSupportChamp,goldblueSupport,blueBans,redTop,redTopChamp,goldredTop,redJungle,redJungleChamp,goldredJungle,redMiddle,redMiddleChamp,goldredMiddle,redADC,redADCChamp,goldredADC,redSupport,redSupportChamp,goldredSupport,redBans,Address,blueTeamLeague,redTeamLeague
0,NALCS,2015,Spring,Season,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,...","[2415, 2415, 2711, 3887, 5068, 6171, 7412, 866...","[[10.82, 'C9 Hai', 'TSM Bjergsen', [], 9229, 8...","[[27.542, 'MID_LANE', 'BASE_TURRET'], [39.269,...","[[36.686, 'MID_LANE'], [29.274, 'MID_LANE']]","[[37.267, None]]",[],[],"[2415, 2415, 2725, 3952, 5336, 6602, 7900, 945...","[[16.529, 'TSM Lustboy', 'C9 Balls', ['C9 Mete...","[[39.23, 'TOP_LANE', 'INNER_TURRET'], [20.681,...",[],"[[17.14, None], [30.934, None], [24.641, None]]",[[29.954]],[],Dyrus,Irelia,"[475, 475, 532, 687, 893, 1058, 1172, 1471, 18...",Santorin,RekSai,"[475, 475, 532, 870, 1049, 1276, 1596, 1815, 2...",Bjergsen,Ahri,"[475, 475, 532, 807, 1102, 1307, 1651, 1950, 2...",WildTurtle,Jinx,"[475, 475, 532, 797, 1127, 1453, 1766, 2044, 2...",Lustboy,Janna,"[515, 515, 583, 726, 897, 1077, 1227, 1381, 15...","['Rumble', 'Kassadin', 'Lissandra']",Balls,Gnar,"[475, 475, 532, 728, 958, 1284, 1526, 1912, 21...",Meteos,Elise,"[475, 475, 532, 898, 1192, 1429, 1819, 2107, 2...",Hai,Fizz,"[475, 475, 552, 842, 1178, 1378, 1635, 1949, 2...",Sneaky,Sivir,"[475, 475, 532, 762, 1097, 1469, 1726, 2112, 2...",LemonNation,Thresh,"[515, 515, 577, 722, 911, 1042, 1194, 1370, 14...","['Tristana', 'Leblanc', 'Nidalee']",http://matchhistory.na.leagueoflegends.com/en/...,NALCS,NALCS


In [ ]:
redTeamLeagues = list()
blueTeamLeagues = list()
wcmatches = leagueoflegends.loc[leagueoflegends['League'] == 'WC']
for row in wcmatches.index:
  redLeague = times_to_liga[leagueoflegends.iloc[row]['redTeamTag']]
  blueLeague = times_to_liga[leagueoflegends.iloc[row]['blueTeamTag']]
  leagueoflegends.at[row, 'redTeamLeague'] = redLeague
  leagueoflegends.at[row, 'blueTeamLeague'] = blueLeague

In [ ]:
leagueoflegends.loc[leagueoflegends['League'] == 'WC'].head(n=1)

,League,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff,goldblue,bKills,bTowers,bInhibs,bDragons,bBarons,bHeralds,goldred,rKills,rTowers,rInhibs,rDragons,rBarons,rHeralds,blueTop,blueTopChamp,goldblueTop,blueJungle,blueJungleChamp,goldblueJungle,blueMiddle,blueMiddleChamp,goldblueMiddle,blueADC,blueADCChamp,goldblueADC,blueSupport,blueSupportChamp,goldblueSupport,blueBans,redTop,redTopChamp,goldredTop,redJungle,redJungleChamp,goldredJungle,redMiddle,redMiddleChamp,goldredMiddle,redADC,redADCChamp,goldredADC,redSupport,redSupportChamp,goldredSupport,redBans,Address,blueTeamLeague,redTeamLeague
6391,WC,2014,Summer,International,EDG,0,1,SSW,44,"[0, 0, 13, -33, 151, -82, 156, 90, -552, 391, ...","[2415, 2415, 2723, 3941, 5324, 6418, 7732, 907...","[[13.729, 'SSW PawN', 'EDG U', ['EDG Clearlove...","[[14.332, 'MID_LANE', 'OUTER_TURRET'], [15.003...",[],"[[8.203, None], [35.336, None]]",[],[],"[2415, 2415, 2710, 3974, 5173, 6500, 7576, 898...","[[7.963, 'EDG Koro1', 'SSW Dandy', ['SSW Loope...","[[43.778, 'MID_LANE', 'NEXUS_TURRET'], [43.215...","[[43.302, 'TOP_LANE'], [40.195, 'BOT_LANE']]","[[21.197, None], [28.368, None], [15.015, None]]","[[34.504], [23.426]]",[],Koro1,Maokai,"[475, 475, 532, 830, 1163, 1364, 1730, 2069, 2...",Clearlove,JarvanIV,"[475, 475, 532, 794, 1011, 1195, 1358, 1578, 1...",U,Orianna,"[475, 475, 551, 816, 1155, 1403, 1753, 2053, 2...",NaMei,Lucian,"[475, 475, 532, 783, 1121, 1444, 1741, 2085, 2...",Fzzf,Thresh,"[515, 515, 576, 718, 874, 1012, 1150, 1286, 14...","['Yasuo', 'Kassadin', 'Alistar']",Looper,Rumble,"[475, 475, 532, 769, 1020, 1385, 1581, 1925, 2...",Dandy,Khazix,"[475, 475, 532, 841, 1011, 1243, 1480, 1798, 2...",PawN,Zilean,"[475, 475, 532, 802, 1093, 1397, 1612, 1848, 2...",imp,Tristana,"[475, 475, 532, 825, 1140, 1400, 1683, 2001, 2...",Mata,Nami,"[515, 515, 582, 737, 909, 1075, 1220, 1409, 15...","['Twitch', 'Ryze', 'Janna']",http://matchhistory.na.leagueoflegends.com/en/...,LPL,LCK


In [ ]:
leagueoflegends.columns

Index(['League', 'Year', 'Season', 'Type', 'blueTeamTag', 'bResult', 'rResult',
       'redTeamTag', 'gamelength', 'golddiff', 'goldblue', 'bKills', 'bTowers',
       'bInhibs', 'bDragons', 'bBarons', 'bHeralds', 'goldred', 'rKills',
       'rTowers', 'rInhibs', 'rDragons', 'rBarons', 'rHeralds', 'blueTop',
       'blueTopChamp', 'goldblueTop', 'blueJungle', 'blueJungleChamp',
       'goldblueJungle', 'blueMiddle', 'blueMiddleChamp', 'goldblueMiddle',
       'blueADC', 'blueADCChamp', 'goldblueADC', 'blueSupport',
       'blueSupportChamp', 'goldblueSupport', 'blueBans', 'redTop',
       'redTopChamp', 'goldredTop', 'redJungle', 'redJungleChamp',
       'goldredJungle', 'redMiddle', 'redMiddleChamp', 'goldredMiddle',
       'redADC', 'redADCChamp', 'goldredADC', 'redSupport', 'redSupportChamp',
       'goldredSupport', 'redBans', 'Address', 'blueTeamLeague',
       'redTeamLeague'],
      dtype='object')

In [ ]:
useful_columns = ['League', 'Year', 'Season', 'Type', 'blueTeamLeague', 'blueTeamTag', 'bResult', 'rResult',
       'redTeamTag', 'redTeamLeague', 'gamelength', 'blueTop', 'blueTopChamp', 'blueJungle', 'blueJungleChamp',
       'blueMiddle', 'blueMiddleChamp', 'blueADC', 'blueADCChamp', 'blueSupport', 'blueSupportChamp', 'blueBans', 
       'redTop', 'redTopChamp', 'redJungle', 'redJungleChamp', 'redMiddle', 'redMiddleChamp', 'redADC', 
       'redADCChamp', 'redSupport', 'redSupportChamp','redBans']

useful_df = leagueoflegends[useful_columns]
useful_df.head()

,League,Year,Season,Type,blueTeamLeague,blueTeamTag,bResult,rResult,redTeamTag,redTeamLeague,gamelength,blueTop,blueTopChamp,blueJungle,blueJungleChamp,blueMiddle,blueMiddleChamp,blueADC,blueADCChamp,blueSupport,blueSupportChamp,blueBans,redTop,redTopChamp,redJungle,redJungleChamp,redMiddle,redMiddleChamp,redADC,redADCChamp,redSupport,redSupportChamp,redBans
0,NALCS,2015,Spring,Season,NALCS,TSM,1,0,C9,NALCS,40,Dyrus,Irelia,Santorin,RekSai,Bjergsen,Ahri,WildTurtle,Jinx,Lustboy,Janna,"['Rumble', 'Kassadin', 'Lissandra']",Balls,Gnar,Meteos,Elise,Hai,Fizz,Sneaky,Sivir,LemonNation,Thresh,"['Tristana', 'Leblanc', 'Nidalee']"
1,NALCS,2015,Spring,Season,NALCS,CST,0,1,DIG,NALCS,38,Cris,Gnar,Impaler,Rengar,Jesiz,Ahri,Mash,Caitlyn,Sheep,Leona,"['Kassadin', 'Sivir', 'Lissandra']",Gamsu,Irelia,Crumbzz,JarvanIV,Shiphtur,Azir,CoreJJ,Corki,KiWiKiD,Annie,"['RekSai', 'Janna', 'Leblanc']"
2,NALCS,2015,Spring,Season,NALCS,WFX,1,0,GV,NALCS,40,Flaresz,Renekton,ShorterACE,Rengar,Pobelter,Fizz,Altec,Sivir,Gleeb,Annie,"['JarvanIV', 'Lissandra', 'Kassadin']",Hauntzer,Sion,Saintvicious,LeeSin,Keane,Azir,Cop,Corki,BunnyFuFuu,Janna,"['Leblanc', 'Zed', 'RekSai']"
3,NALCS,2015,Spring,Season,NALCS,TIP,0,1,TL,NALCS,41,Rhux,Irelia,Rush,JarvanIV,XiaoWeiXiao,Leblanc,Apollo,Sivir,Adrian,Thresh,"['Annie', 'Lissandra', 'Kassadin']",Quas,Gnar,IWDominate,Nunu,Fenix,Lulu,KEITH,KogMaw,Xpecial,Janna,"['RekSai', 'Rumble', 'LeeSin']"
4,NALCS,2015,Spring,Season,NALCS,CLG,1,0,T8,NALCS,35,Benny,Gnar,Xmithie,JarvanIV,Link,Lissandra,Doublelift,Tristana,aphromoo,Janna,"['Irelia', 'Pantheon', 'Kassadin']",CaliTrlolz8,Sion,Porpoise8,RekSai,Slooshi8,Lulu,Maplestreet8,Corki,Dodo8,Annie,"['Rumble', 'Sivir', 'Rengar']"


Essa célula exporta o primeiro dataframe que será usado na análise

In [ ]:
#useful_df.to_csv('Dataframe_Filtrado.csv', index=False)

# Criação de novo dataframe

Nessa seção é criado um dataframe que acreditamos ser importante para análise, ele contem a taxa de vitória e escolha de cada campeão 

In [ ]:
Allchampions = list(leagueoflegends['blueTopChamp'].unique())
Allchampions += list(leagueoflegends['blueJungleChamp'].unique())
Allchampions += list(leagueoflegends['blueMiddleChamp'].unique())
Allchampions += list(leagueoflegends['blueADCChamp'].unique())
Allchampions += list(leagueoflegends['blueSupportChamp'].unique())
Allchampions += list(leagueoflegends['redTopChamp'].unique())
Allchampions += list(leagueoflegends['redJungleChamp'].unique())
Allchampions += list(leagueoflegends['redMiddleChamp'].unique())
Allchampions += list(leagueoflegends['redADCChamp'].unique())
Allchampions += list(leagueoflegends['redSupportChamp'].unique())
Allchampions = np.array(Allchampions)
Allchampions = np.unique(Allchampions)

In [ ]:
bChampLanes = ['blueTopChamp','blueJungleChamp','blueMiddleChamp','blueADCChamp','blueSupportChamp']
rChampLanes = ['redTopChamp','redJungleChamp','redMiddleChamp','redADCChamp','redSupportChamp']
blueWin = leagueoflegends[leagueoflegends['bResult'] == 1]
redWin = leagueoflegends[leagueoflegends['rResult'] == 1]
blueWin = blueWin[bChampLanes]
redWin = redWin[rChampLanes]
nMatches = leagueoflegends.shape[0]

WinRates = pd.DataFrame()
for champ in Allchampions:

  redchampionw = redWin.eq(champ).sum().sum()
  bluechampionw = blueWin.eq(champ).sum().sum()

  ChampRate = leagueoflegends.eq(champ).sum().sum()
  PickRate = ChampRate/nMatches

  champw = redchampionw + bluechampionw
  WinRates = WinRates.append(pd.Series([champ,champw/ChampRate,PickRate,ChampRate]), ignore_index=True)

WinRates = WinRates.rename(columns={0:'champ',1:'wr',2:'pr',3:'npick'})
WinRates = WinRates.set_index('champ')
WinRates = WinRates.sort_values(by='npick',ascending=0)


,wr,pr,npick
champ,,,
Gragas,0.524462,0.335302,2555.0
RekSai,0.483262,0.305774,2330.0
Braum,0.483688,0.277559,2115.0
Elise,0.505569,0.270997,2065.0
Thresh,0.522621,0.252362,1923.0
Maokai,0.509677,0.244094,1860.0
Lucian,0.475679,0.232021,1768.0
Corki,0.488532,0.228871,1744.0
Sivir,0.524970,0.220735,1682.0


Essa célula exporta o segundo dataframe que será usado na análise

In [ ]:
#WinRates.to_csv('Winrates.csv', index=True)